In [ ]:
from sentence_transformers import InputExample
import pandas as pd
df = pd.read_csv('pair_score.csv')

train_samples = [
    InputExample(texts=[row["sentence1"], row["sentence2"]], label=row['score']) for index, row in df.iterrows()
]

In [ ]:
df1 = pd.read_csv('test_pair_score.csv')

test_samples = [
    InputExample(texts=[row["sentence1"], row["sentence2"]], label=row['score']) for index, row in df1.iterrows()
]

In [ ]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from sentence_transformers import CrossEncoder
import torch

In [ ]:
model = CrossEncoder('BAAI/bge-reranker-v2-m3', num_labels=1)

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ADMIN\.cache\huggingface\hub\models--BAAI--bge-reranker-v2-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

In [ ]:
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=2)

In [ ]:
from torch.utils.data import DataLoader

from sentence_transformers import LoggingHandler, util
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import 
from sentence_transformers.evaluation import SequentialEvaluator
from sentence_transformers.readers import InputExample
import math

In [ ]:
accuracy_evaluator = CESoftmaxAccuracyEvaluator.from_input_examples(test_samples, name="AllNLI-test")
f1_evaluator = CEF1Evaluator.from_input_examples(test_samples, name="AllNLI-test")
evaluator = SequentialEvaluator([accuracy_evaluator, f1_evaluator])

In [ ]:
num_epochs = 2

In [ ]:
model.fit(
    train_dataloader=train_dataloader,
    evaluator=evaluator,
    epochs=num_epochs,
    warmup_steps= math.ceil(len(train_dataloader) * num_epochs * 0.1),
    output_path='./results',
)

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/68941 [00:00<?, ?it/s]